<a href="https://colab.research.google.com/github/rodmart21/Sports_analytics/blob/main/ai_scout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [14]:
import requests
import pandas as pd

from datetime import datetime

from openai import OpenAI
from bs4 import BeautifulSoup

client = OpenAI(
    api_key="" # PUT YOUR API KEY here
)

### Sacamos

In [15]:
# Lamine Yamal
player_name = 'Lamine Yamal'
url = 'https://fbref.com/en/players/82ec26c1/Lamine-Yamal'
attrs = 'scout_summary_AM'

# # Nathan Broadhead
# player_name = 'Nathan Broadhead'
# url = 'https://fbref.com/en/players/43309491/Nathan-Broadhead'
# attrs = 'scout_summary_AM'


df = pd.read_html(
    url,
    attrs={'id': attrs} # Change the id & class to correspond to the table
)[0]

In [16]:
df  # El per90 nos da las gráficas en función de los 90 minutos de juego, de forma que no influye que un jugador haya podido jugar más o menos.
# El percentile indica el porcentaje de jugadores respecto a los cuales ha rendido mejor. Percentil 90 significa que ha rendido mejor que el 90% de los jugadores.

,Statistic,Per 90,Percentile
0,Non-Penalty Goals,0.17,33.0
1,npxG: Non-Penalty xG,0.23,59.0
2,Shots Total,2.58,70.0
3,Assists,0.23,65.0
4,xAG: Exp. Assisted Goals,0.27,84.0
5,npxG + xAG,0.50,74.0
6,Shot-Creating Actions,4.51,73.0
7,NaN,NaN,NaN
8,Passes Attempted,45.22,73.0
9,Pass Completion %,78.8%,70.0


In [17]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [18]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()

In [19]:
# df.columns = df.columns.droplevel(0)
df = df.dropna(subset='Statistic')

### Prompt sencillo en el que le decimos las tres cosas fundamentales que queremos que tenga nuestro report.

In [20]:
prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team >
"""

In [21]:
print(prompt)



I need you to create a scouting report on Lamine Yamal. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: Lamine Yamal
Position: FW
Age: 16
Team: Barcelona

|    | Statistic                | Per 90   |   Percentile |
|---:|:-------------------------|:---------|-------------:|
|  0 | Non-Penalty Goals        | 0.17     |           33 |
|  1 | npxG: Non-Penalty xG     | 0.23     |           59 |
|  2 | Shots Total              | 2.58     |           70 |
|  3 | Assists                  | 0.23     |           65 |
|  4 | xAG: Exp. Assisted Goals | 0.27     |           84 |
|  5 | npxG + xAG               | 0.50     |           74 |
|  6 | Shot-Creating Actions    | 4.51     |           73 |
|  8 | Passes Attempted         | 45.22    |           73 |
|  9 | Pass Completion %        | 78.8%    |           70 |
| 10 | Progressive Passes       | 3.31     |           35 |
| 11 | Progressive Carries      | 5.00     |           82 |


### Este modelo nos da error 404, seguramente con nuestra API key no tengamos acceso a él.

In [33]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-turbo` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

### Este otro modelo nos da error 429, hemos gastado la cuota que podemos gastar, habría que pagar más pasta.

In [32]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## En esta página tienes info sobre todos los fallos que puedes obtener al conectarte a la API de ChatGPT. https://platform.openai.com/docs/guides/error-codes/api-errors

## No puedo acabar porque no se me conecta a ninguna de estas APIs del Chat.

In [34]:
print(response.choices[0].message.content)

AttributeError: 'Response' object has no attribute 'choices'

In [ ]:
# save the scouting report to a new markdown file
with open(f"/Users/mckayjohns/Documents/GitHub/youtube-videos/{player_name}_scouting_report.md", "w") as f:
    f.write(response.choices[0].message.content)